In [1]:
import pandas as pd;
import numpy as np;
#import serial, time 
import json

In [ ]:
#!pip install --upgrade pandas

In [ ]:
# connect to Track
ser = serial.Serial('/dev/ttyUSB0', 9600, timeout = None)
print(ser.name)

In [ ]:
# csv or excel
# name, car number, den, category

# "Race number" car number" x4, "lane" for heats

In [2]:
# Read from CSV to load Races
races = pd.read_csv('Races.csv')
races = races.set_index('Race Number')
#races.head()

In [5]:
#races.loc[2,'Lane 2'] = 1
races.head(15)

,Lane 1,Lane 2,Lane 3,Lane 4
Race Number,,,,
1,1,9,8,7
2,2,1,9,8
3,3,2,1,9
4,4,3,2,1
5,5,4,3,2
6,6,5,4,3
7,7,6,5,4
8,8,7,6,5
9,9,8,7,6


In [6]:
# Read list of racers and cars from CSV

racers = pd.read_csv('Racers.csv')
racers = racers.set_index('Car Number')
racers.head()

,Name,Den,Category
Car Number,,,
1,"Barthelmas, Aiden",2 - WOLF,Pack
2,"Blackler, Caleb",4 - WEBLOS I,Pack
3,"Bracken, Basil",5 - WEBLOS II,Pack
4,"Brockenbrough, Myles",3 - BEAR,Pack
5,"Capone, Declan",1 - TIGER,Pack


In [7]:
# Set Race
current_race = 0

# Start of the Race process

In [9]:
# resultsdf = pd.DataFrame(columns=["Race","Lane","Car","Time","Scout","Den"])
#resultsdf = pd.DataFrame(columns=["Race","Lane","Car","Time"])

resultsdf = pd.DataFrame(columns=['race', 'lane', 'car', 'name', 'time', 'place', 'den', 'category'])
resultsdf.head()

,race,lane,car,name,time,place,den,category


In [10]:
# set current race if needed
#current_race = 1

In [11]:
print(current_race)

1


In [12]:
# show curret race
current_contestants = races.loc[current_race]
print(races.loc[current_race])

Lane 1    1
Lane 2    9
Lane 3    8
Lane 4    7
Name: 1, dtype: int64


In [49]:
# create Race Output for Upcoming race
# for race, build details for lane
# get lane 1 car
race = {}
lane = 1
for carNum in races.loc[current_race+1]:
    name = racers.loc[carNum,'Name']
    #print(lane, carNum, name)
    details = {'car': str(carNum), 'name': name}
    trackName = 'lane' + str(lane)
    race[trackName] = details
    lane = lane +1

nextup = {'nextup': race}
with open('next.json', 'w') as outfile:
    json.dump(nextup, outfile)


In [13]:
new1 = current_contestants[current_contestants == 0]
print(new1)

Series([], Name: 1, dtype: int64)


In [ ]:
print(new1.index.values)

## Run Race


In [ ]:
# Turn on all Lanes   om# is used to turn off track
command = 'om0'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
returnmessage = ser.readline()
print(returnmessage.decode())

In [ ]:
# Reset Track if needed
#Reset Track
command = 'r'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)

In [ ]:
# Turn off tracks that are not being used

for x in (new1.index.values):
    lanenum = (x[4:5])
    command = 'om' + lanenum +'\r\n'
    print(command)
    #newcommand = command.encode('ascii')
    #ser.write(newcommand)
    #returnmessage = ser.readline()
    #print(returnmessage.decode())

In [ ]:
# Get Race Results
# get race results
command = 'rg'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
results = ser.readline()
laneresults = results.decode().split()
print(laneresults)


In [50]:
laneresults = ['1=1.1704a', '2=5.4159b', '3=3.5462c', '4=3.7246d']
print(laneresults)

['1=1.1704a', '2=5.4159b', '3=3.5462c', '4=3.7246d']


# Force finish race 

## Write Results to results df


In [51]:
race = {}
for laneresult in laneresults:
    lane = laneresult[0]
    car = races.loc[int(current_race),'Lane '+lane]
    time = float(laneresult[2:8])
    name = racers.loc[car,'Name']
    den = racers.loc[car,'Den']
    category = racers.loc[car,'Category']
    place = laneresult[8:9]
    
    # Create results json
    details = {'race' : current_race , 'lane' : int(lane), 'car': str(car), 'name': name, 'time': time, 'place': place, 'den': den, 'category': category} 
    trackName = 'lane' + str(lane)
    race[trackName] = details
    
 
    # add den, category
    resultsdf = resultsdf.append(details, ignore_index=True)
    print(details)
   
current_race = current_race +1

{'den': '3 - BEAR', 'place': 'a', 'name': 'Brockenbrough, Myles', 'lane': 1, 'time': 1.1704, 'car': '4', 'race': 4, 'category': 'Pack'}
{'den': '5 - WEBLOS II', 'place': 'b', 'name': 'Bracken, Basil', 'lane': 2, 'time': 5.4159, 'car': '3', 'race': 4, 'category': 'Pack'}
{'den': '4 - WEBLOS I', 'place': 'c', 'name': 'Blackler, Caleb', 'lane': 3, 'time': 3.5462, 'car': '2', 'race': 4, 'category': 'Pack'}
{'den': '2 - WOLF', 'place': 'd', 'name': 'Barthelmas, Aiden', 'lane': 4, 'time': 3.7246, 'car': '1', 'race': 4, 'category': 'Pack'}


In [52]:
## update or rewrite Json for current race.  
currentRace = {'currentRace': race}
with open('current.json', 'w') as outfile:
    json.dump(currentRace, outfile)

In [43]:
resultsdf.tail(100)

,race,lane,car,name,time,place,den,category
0,1,1,1,"Barthelmas, Aiden",2.1704,a,2 - WOLF,Pack
1,1,2,9,"Edmonds, Harvey",2.4159,b,2 - WOLF,Pack
2,1,3,8,"Donnarumma, Liam",2.5462,c,1 - TIGER,Pack
3,1,4,7,"Debow, Anderson",2.7246,d,4 - WEBLOS I,Pack
4,2,1,2,"Blackler, Caleb",3.1704,a,4 - WEBLOS I,Pack
5,2,2,1,"Barthelmas, Aiden",3.4159,b,2 - WOLF,Pack
6,2,3,9,"Edmonds, Harvey",4.5462,c,2 - WOLF,Pack
7,2,4,8,"Donnarumma, Liam",5.7246,d,1 - TIGER,Pack
8,3,1,3,"Bracken, Basil",1.1704,a,5 - WEBLOS II,Pack
9,3,2,2,"Blackler, Caleb",5.4159,b,4 - WEBLOS I,Pack


In [47]:
# Get top cars
standings = resultsdf.groupby('car') \
    .apply(lambda x: x.drop([x['time'].idxmax()]))\
    .rename_axis(['time','time'])\
    .groupby('car')\
    .agg({'car':'size', 'time':'mean'}) \
    .rename(columns={'car':'Races Completed','time':'Average Time'}) \
    .sort_values('Average Time', ascending=True)


#standings = standings.sort_values('Average Time', ascending=True)
print(standings.iloc[0:4])

     Races Completed  Average Time
car                               
8                  1       2.54620
1                  2       2.79315
9                  2       3.07025
2                  1       3.17040


In [36]:
standings.iloc[0:5]

,Races Completed,Average Time
car,,
1,1,2.1704
9,1,2.4159
8,1,2.5462


## Calculate leaders based on average time

In [ ]:
#avg = resultsdf.groupby('Car')['Time'].mean()
#type(avg)

In [19]:
avg = resultsdf.groupby('car') \
       .agg({'car':'size', 'time':'mean'}) \
       .rename(columns={'car':'Races Completed','time':'Average Time'}) \
       .reset_index()
#avg = avg.set_index('Car')

In [28]:
avg.head()

,car,Races Completed,Average Time
0,1,1,2.1704
1,7,1,2.7246
2,8,1,2.5462
3,9,1,2.4159


In [ ]:
leaders = avg.sort_values(by='Average Time').reset_index()

In [ ]:
leaders.head()

## Export Results

In [ ]:
with open('leaders.json','w') as lj:
    leaders.iloc[0:10].to_json(lj)

# Extra commands

In [ ]:
leaderdf.head()

In [ ]:
# Find Car by race and lane
car = racedf.loc[1,'lane1']
print (car)

In [ ]:
# delete race result or duplicate race result

In [ ]:
# write out rsults df and load from csv

In [ ]:
def serialwrite(code):
    command = code +'\r\n'
    #newcommand = command.encode('ascii')
    #ser.write(newcommand)
    #results = ser.readline()
    #laneresults = results.decode().split()
    #print(laneresults)
    return results

serialwrite('rg')

In [ ]:
currentrace['currentRace']['lane1']['time'] ='3.012'
lastRace = currentrace['currentRace']
type(lastRace)